In [ ]:
# ! pip install evaluate
# ! pip install bert_score
# ! pip install sacrebleu
# ! pip install nltk
# ! pip install rouge_score
# ! pip install torchmetrics
# # for BLEURT refer to https://github.com/google-research/bleurt 

# Load annotations

In [1]:
import json
import pprint 
import numpy as np
from evaluate import load

pp = pprint.PrettyPrinter(indent=4)
img_prompt_respond_file = '../data/memecap/filtered_meme_configs_5_attributes_memecap.json'
# Load predictions
predictions = []
with open(img_prompt_respond_file, 'r', encoding='utf-8') as json_file:
    predictions = json.load(json_file)
len(predictions), predictions[0]['meaning of the meme'], predictions[0]['image_dir'].split('/')[-1]

/home/shiling/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-30 15:26:54.296739: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 15:26:54.316714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 15:26:54.328225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 15:26

(464,
 "- The meme humorously suggests that a wife, after losing her memory, fell in love with and decided to remarry her own husband, indicating his unchanging worth and likeability as a partner, paralleled by the character's proud declaration of worthiness regarding his hammer.",
 'memes_d079np.png')

# Load gtound truth 

In [2]:
gt = []
with open('../data/memecap/meme-cap-main/data/memes-test.json', 'r', encoding='utf-8') as json_file:
    gt = json.load(json_file)
len(gt), gt[0]['meme_captions'], gt[0]['img_fname']
# concatenate all the references from the ground truth.
memecap_predictions_concatenated = []
memecap_processed_gt_concatenated = []
# copy the predictions to match the number of references
memecap_predictions_extended = []
memecap_processed_gt_extended = []
# Do nothing. One-prediction to multi-reference
memecap_predictions_multi_ref = []
memecap_processed_gt_multi_ref = []

for idx, meme_conf in enumerate(predictions):
    for meme in gt:
        if meme_conf['image_dir'].split('/')[-1] == meme['img_fname']:
            memecap_predictions_concatenated.append(meme_conf['meaning of the meme'])
            memecap_processed_gt_concatenated.append(''.join(meme['meme_captions']))
            
            memecap_predictions_extended += [meme_conf['meaning of the meme']] * len(meme['meme_captions'])
            memecap_processed_gt_extended += meme['meme_captions']

            memecap_predictions_multi_ref.append(meme_conf['meaning of the meme'])
            memecap_processed_gt_multi_ref.append(meme['meme_captions'])
len(memecap_processed_gt_concatenated), len(memecap_processed_gt_extended), len(memecap_predictions_extended), len(memecap_predictions_multi_ref), len(memecap_processed_gt_multi_ref)

(464, 1567, 1567, 464, 464)

# BLEURT
## Github repo
https://github.com/google-research/bleurt

In [5]:
def calculate_the_best_matches(target_scores, sample_size, multi_ref): 
    # Calculate the score for the best matches 
    restored_scores = []
    last_idx = 0
    for i in range(sample_size):
        restored_scores.append(target_scores[last_idx : last_idx + len(multi_ref[i])])
        last_idx += len(multi_ref[i]) 
    # print(restored_scores)
    maximized = [ max(ite) for ite in restored_scores]
    print('The average score for the best matches:', np.mean(maximized))

In [4]:
from bleurt import score

checkpoint = "/home/shiling/git/bleurt/BLEURT-20"
# references = ["This is a test.", "This is a test."]
# candidates = ["This is a test.", "This is the test."]
# [0.9881654977798462, 0.7926645278930664]

scorer = score.BleurtScorer(checkpoint)
# Extended predictions
scores = scorer.score(references=memecap_processed_gt_extended, candidates=memecap_predictions_extended)
print('Extended version:', np.mean(scores))

calculate_the_best_matches(scores, len(memecap_predictions_multi_ref), memecap_processed_gt_multi_ref)

# Concatenated references
scores = scorer.score(references=memecap_processed_gt_concatenated, candidates=memecap_predictions_concatenated)
print('Concatenated version:', np.mean(scores))


INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.


INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint BLEURT-20


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Extended version: 0.47095482013335105
The average score for the best matches: 0.5254725490793072
Concatenated version: 0.47439709543411074


## Huggingface interface
https://huggingface.co/spaces/evaluate-metric/bleurt/blob/main/README.md

In [15]:
bleurt_scorer = load("bleurt", module_type="metric", checkpoint="BLEURT-20")
references = ["This is a test.", "This is a test."]
candidates = ["This is a test.", "This is the test."]
scores = bleurt_scorer.compute(predictions=candidates, references=references)
print('Example:', np.mean(scores['scores']))
# Extended predictions
scores = bleurt_scorer.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended)
print('Extended version:', np.mean(scores['scores']))

# Concatenated references
scores = bleurt_scorer.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print('Concatenated version:', np.mean(scores['scores']))

INFO:tensorflow:Reading checkpoint /home/shiling/.cache/huggingface/metrics/bleurt/default/downloads/extracted/13db1fc9d295585583f8a5538b232c82f61e73738f37827f3c4fc14396e60785/bleurt-base-128.


INFO:tensorflow:Reading checkpoint /home/shiling/.cache/huggingface/metrics/bleurt/default/downloads/extracted/13db1fc9d295585583f8a5538b232c82f61e73738f37827f3c4fc14396e60785/bleurt-base-128.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint bert_custom


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Example: 0.8682224154472351
Extended version: -0.6088926426224748
Concatenated version: -0.5713593499445967


# BERTscore

In [3]:
bertscore = load("bertscore")

In [6]:
# Extended predictions
results = bertscore.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended, lang="en")
print(results.keys())
print(np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

calculate_the_best_matches(results['f1'], len(memecap_predictions_multi_ref), memecap_processed_gt_multi_ref)

# Concatenated references
results = bertscore.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated, lang="en")
print(np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

# One-prediction to multi-reference
results = bertscore.compute(predictions=memecap_predictions_multi_ref, references=memecap_processed_gt_multi_ref, lang="en")
print(np.mean(results['precision']), np.mean(results['recall']), np.mean(results['f1']))

dict_keys(['precision', 'recall', 'f1', 'hashcode'])
0.8596986218782923 0.8784849302891948 0.8689020508858681
The average score for the best matches: 0.879493215859964
0.8675926138871702 0.8604510788773668 0.8639449427097008
0.8698499315771563 0.8925056580839485 0.879493215859964


# ChrF

## Huggingface interface

In [8]:
# prediction = ["The relationship between cats and dogs is not exactly friendly.", "a good bookshop is just a genteel black hole that knows how to read."]
# reference = [["The relationship between dogs and cats is not exactly friendly.", ], ["A good bookshop is just a genteel Black Hole that knows how to read."]]
chrf = load("chrf")

# {'score': 84.64214891738334, 'char_order': 6, 'word_order': 0, 'beta': 2}

# Extended predictions
results = chrf.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended)
print(results)
# Concatenated predictions
results = chrf.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated)
print(results)

In [11]:
# # one prediction to multiple references -- Huggingface
# # ValueError: ChrF, as implemented by sacrebleu, requires the same number of references for each prediction
# results = chrf.compute(predictions=memecap_predictions_multi_ref, references=memecap_processed_gt_multi_ref)
# print(results)

## Torchmetrics interface

In [12]:
# one prediction and multiple references
from torchmetrics.text import CHRFScore
# preds = ['the cat is on the mat']
# target = [['there is a cat on the mat', 'a cat is on the mat']]
chrf = CHRFScore()
chrf(memecap_predictions_multi_ref, memecap_processed_gt_multi_ref)

tensor(0.2883)

In [13]:
# Extended predictions
chrf(memecap_predictions_extended, memecap_processed_gt_extended)

tensor(0.2450)

In [14]:
# Concatenated references
chrf(memecap_predictions_concatenated, memecap_processed_gt_concatenated)

tensor(0.2077)

# ROUGE

In [18]:
rouge = load('rouge')
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]

# {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [19]:
# Extended predictions
results = rouge.compute(predictions=memecap_predictions_extended,
                        references=memecap_processed_gt_extended)
print(results)

# Concatenated references
results = rouge.compute(predictions=memecap_predictions_concatenated,
                        references=memecap_processed_gt_concatenated)
print(results)

# one prediction VS multiple references
results = rouge.compute(predictions=memecap_predictions_multi_ref, 
                        references=memecap_processed_gt_multi_ref)
print(results)

{'rouge1': 0.21358079742963504, 'rouge2': 0.0416616902071557, 'rougeL': 0.17265236393343997, 'rougeLsum': 0.1726669393742032}


# bleu

In [22]:
# predictions = ["hello there general kenobi", "foo bar foobar"]
# references = [
#     ["hello there general kenobi", "hello there !"],
#     ["foo bar foobar"]
# ]
bleu =load("bleu")

# {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.1666666666666667, 'translation_length': 7, 'reference_length': 6}

# Extended predictions
bleu =load("bleu")
results = bleu.compute(predictions=memecap_predictions_extended, 
                       references=memecap_processed_gt_extended, max_order=4)
print(results)


# Concatenated references
results = bleu.compute(predictions=memecap_predictions_concatenated, 
                       references=memecap_processed_gt_concatenated)
print(results)

In [25]:
# one prediction VS multiple references
from nltk.tokenize import word_tokenize
results = bleu.compute(predictions=memecap_predictions_multi_ref, 
                       references=memecap_processed_gt_multi_ref, tokenizer=word_tokenize)
print(results)

{'bleu': 0.03673635280307039, 'precisions': [0.29231815091774305, 0.06801552459120698, 0.016127974824624663, 0.005679897220907431], 'brevity_penalty': 1.0, 'length_ratio': 2.7467323035138347, 'translation_length': 16181, 'reference_length': 5891}


# Justify that the sentence length has an influence on the final score for n-gram based metrics

In [2]:
predictions = ["- The meme humorously suggests that a wife, after losing her memory, fell in love with and decided to remarry her own husband, indicating his unchanging worth and likeability as a partner, paralleled by the character's proud declaration of worthiness regarding his hammer.", 
               'The meme humorously suggests that the TV show "The Simpsons" predicted real-life events involving Greta Thunberg and Donald Trump.']

ground_truth = [[
    'Husband feels great after having their wife fall in love with him again after getting amnesia.',
    'The meme poster feels happy for the person who make his wife remember their love even after she forgot all.',
    'meme poster is conveying they feel like thor when a woman says to marrying them '], [
    "The Simpsons was correct about it's use of Trump and Greta Thurnberg. ",
    'The Simpsons is able to predict so many real life situation including Trump and Greta Thurnburg.'
    ]]

len(predictions[0]), len(predictions[1])

(272, 130)

In [3]:
gt = ground_truth

# concatenate all the references from the ground truth.
memecap_predictions_concatenated = []
memecap_processed_gt_concatenated = []
# copy the predictions to match the number of references
memecap_predictions_extended = []
memecap_processed_gt_extended = []
# Do nothing. One-prediction to multi-reference
memecap_predictions_multi_ref = predictions
memecap_processed_gt_multi_ref = gt

for idx, predic in enumerate(predictions):
    memecap_predictions_concatenated.append(predic)
    memecap_processed_gt_concatenated.append(''.join(gt[idx]))
    
    memecap_predictions_extended += [predic] * len(gt[idx])
    memecap_processed_gt_extended += gt[idx]

len(memecap_processed_gt_concatenated), len(memecap_processed_gt_extended), len(memecap_predictions_extended), len(memecap_predictions_multi_ref), len(memecap_processed_gt_multi_ref)

(2, 5, 5, 2, 2)

In [5]:
## BLEURT
from bleurt import score
checkpoint = "/home/shiling/git/bleurt/BLEURT-20"
bleurt_scorer = score.BleurtScorer(checkpoint)

# Extended predictions
scores = bleurt_scorer.score(references=memecap_processed_gt_extended, candidates=memecap_predictions_extended)
print('Extended version:', scores)

# Calculate the score for the best matches 
restored_scores = []
last_idx = 0
for i in range(len(predictions)):
    restored_scores.append(scores[last_idx : last_idx + len(gt[i])])
    last_idx += len(gt[i])
print(restored_scores)
maximized = [ max(ite) for ite in restored_scores]
print('The average score for the best matches:', np.mean(maximized))

# Concatenated references
scores = bleurt_scorer.score(references=memecap_processed_gt_concatenated, candidates=memecap_predictions_concatenated)
print('Concatenated version:', scores)

INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.


INFO:tensorflow:Reading checkpoint /home/shiling/git/bleurt/BLEURT-20.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Will load checkpoint BLEURT-20


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Extended version: [0.3678780794143677, 0.4148254990577698, 0.34403014183044434, 0.5515329837799072, 0.6498597264289856]
[[0.3678780794143677, 0.4148254990577698, 0.34403014183044434], [0.5515329837799072, 0.6498597264289856]]
The average score for the best matches: 0.5323426127433777
Concatenated version: [0.4250648021697998, 0.6123998165130615]


In [10]:
## BERTscore
bertscore = load("bertscore")
# Extended predictions
results = bertscore.compute(predictions=memecap_predictions_extended, references=memecap_processed_gt_extended, lang="en")
print('Extended version:', (results['precision']), (results['recall']), (results['f1']))

# Concatenated references
results = bertscore.compute(predictions=memecap_predictions_concatenated, references=memecap_processed_gt_concatenated, lang="en")
print('Concatenated version:', (results['precision']), (results['recall']), (results['f1']))

# One-prediction to multi-reference
results = bertscore.compute(predictions=memecap_predictions_multi_ref, references=memecap_processed_gt_multi_ref, lang="en")
print('Multi-ref version:', (results['precision']), (results['recall']), (results['f1']))

/home/shiling/anaconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extended version: [0.8494038581848145, 0.8452954292297363, 0.8215376734733582, 0.8836065530776978, 0.8966962099075317] [0.8845654726028442, 0.8777951598167419, 0.8346765637397766, 0.9020423293113708, 0.9036669135093689] [0.8666281700134277, 0.8612387776374817, 0.8280550241470337, 0.8927293419837952, 0.9001681208610535]
Concatenated version: [0.8486756682395935, 0.8898381590843201] [0.8569967746734619, 0.8870041370391846] [0.8528159260749817, 0.8884188532829285]
Multi-ref version: [0.8494038581848145, 0.8966962099075317] [0.8845654726028442, 0.9036669135093689] [0.8666281700134277, 0.9001681208610535]


In [14]:
## CHrF
chrf = load("chrf")
print("sample 1:")
# Extended predictions
results = chrf.compute(predictions=memecap_predictions_extended[:2], references=memecap_processed_gt_extended[:2])
print('Extended version:', results)
# Concatenated references
results = chrf.compute(predictions=[memecap_predictions_concatenated[0]], references=[memecap_processed_gt_concatenated[0]])
print('Concatenated version:', results)

print("sample 2:")
# Extended predictions
results = chrf.compute(predictions=memecap_predictions_extended[2:], references=memecap_processed_gt_extended[2:])
print('Extended version:', results)
# Concatenated references
results = chrf.compute(predictions=[memecap_predictions_concatenated[1]], references=[memecap_processed_gt_concatenated[1]])
print('Concatenated version:', results)

sample 1:
Extended version: {'score': 26.928533615727623, 'char_order': 6, 'word_order': 0, 'beta': 2}
Concatenated version: {'score': 28.54676238027791, 'char_order': 6, 'word_order': 0, 'beta': 2}
sample 2:
Extended version: {'score': 31.25069471750937, 'char_order': 6, 'word_order': 0, 'beta': 2}
Concatenated version: {'score': 29.486823665744826, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [21]:
## ROUGE
rouge = load('rouge')
print("sample 1:")
# Extended predictions
results = rouge.compute(predictions=memecap_predictions_extended[:2],
                        references=memecap_processed_gt_extended[:2])
print('Extended version:', results)

# Concatenated reference
results = rouge.compute(predictions=[memecap_predictions_concatenated[0]],
                        references=[memecap_processed_gt_concatenated[0]])
print('Concatenated version:', results)

# One-prediction to multi-reference
results = rouge.compute(predictions=[memecap_predictions_multi_ref[0]],
                        references=[memecap_processed_gt_multi_ref[0]])
print('Multi-ref version:', results)

print("sample 2:")
# Extended predictions
results = rouge.compute(predictions=memecap_predictions_extended[2:],
                        references=memecap_processed_gt_extended[2:])
print('Extended version:', results)

# Concatenated reference
results = rouge.compute(predictions=[memecap_predictions_concatenated[1]],
                        references=[memecap_processed_gt_concatenated[1]])
print('Concatenated version:', results)

# One-prediction to multi-reference
results = rouge.compute(predictions=[memecap_predictions_multi_ref[1]],
                        references=[memecap_processed_gt_multi_ref[1]])
print('Multi-ref version:', results)

sample 1:
Extended version: {'rouge1': 0.2128060263653484, 'rouge2': 0.051481161921196436, 'rougeL': 0.131288673661555, 'rougeLsum': 0.131288673661555}
Concatenated version: {'rouge1': 0.2553191489361702, 'rouge2': 0.06521739130434782, 'rougeL': 0.1276595744680851, 'rougeLsum': 0.1276595744680851}
Multi-ref version: {'rouge1': 0.22222222222222227, 'rouge2': 0.07017543859649122, 'rougeL': 0.13559322033898305, 'rougeLsum': 0.13559322033898305}
sample 2:
Extended version: {'rouge1': 0.26512248926042026, 'rouge2': 0.060721062618595834, 'rougeL': 0.18768141181934286, 'rougeLsum': 0.18768141181934286}
Concatenated version: {'rouge1': 0.32653061224489793, 'rouge2': 0.0851063829787234, 'rougeL': 0.24489795918367346, 'rougeLsum': 0.24489795918367346}
Multi-ref version: {'rouge1': 0.38888888888888884, 'rouge2': 0.11764705882352941, 'rougeL': 0.2777777777777778, 'rougeLsum': 0.2777777777777778}


In [23]:
## BLEU
bleu =load("bleu")
print('Sample 1')
# Extended predictions
results = bleu.compute(predictions=memecap_predictions_extended[:2], 
                       references=memecap_processed_gt_extended[:2], max_order=4)
print('Extended version:', results)

# Concatenated reference
results = bleu.compute(predictions=[memecap_predictions_concatenated[0]], 
                       references=[memecap_processed_gt_concatenated[0]], max_order=4)
print('Concatenated version:', results)

# One-prediction to multi-reference
results = bleu.compute(predictions=[memecap_predictions_multi_ref[0]], 
                       references=[memecap_processed_gt_multi_ref[0]], max_order=4)
print('Multi-ref version:', results)

print('Sample 2')
# Extended predictions
results = bleu.compute(predictions=memecap_predictions_extended[2:], 
                       references=memecap_processed_gt_extended[2:], max_order=4)
print('Extended version:', results)

# Concatenated reference
results = bleu.compute(predictions=[memecap_predictions_concatenated[1]], 
                       references=[memecap_processed_gt_concatenated[1]], max_order=4)
print('Concatenated version:', results)

# One-prediction to multi-reference
results = bleu.compute(predictions=[memecap_predictions_multi_ref[1]], 
                       references=[memecap_processed_gt_multi_ref[1]], max_order=4)
print('Multi-ref version:', results)

Sample 1
Extended version: {'bleu': 0.0, 'precisions': [0.14583333333333334, 0.031914893617021274, 0.010869565217391304, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.526315789473684, 'translation_length': 96, 'reference_length': 38}
Concatenated version: {'bleu': 0.0, 'precisions': [0.25, 0.06382978723404255, 0.021739130434782608, 0.0], 'brevity_penalty': 0.9010751057212905, 'length_ratio': 0.9056603773584906, 'translation_length': 48, 'reference_length': 53}
Multi-ref version: {'bleu': 0.0, 'precisions': [0.25, 0.06382978723404255, 0.021739130434782608, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 3.2, 'translation_length': 48, 'reference_length': 15}
Sample 2
Extended version: {'bleu': 0.0, 'precisions': [0.16304347826086957, 0.02247191011235955, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.0444444444444443, 'translation_length': 92, 'reference_length': 45}
Concatenated version: {'bleu': 0.0, 'precisions': [0.3181818181818182, 0.047619047619047616, 0.0, 0.0], 'brevity_pen